<a href="https://colab.research.google.com/github/guyyariv/AI-Beat-Maker/blob/master/drums_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

In [34]:
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

In [ ]:
!pip install torch fastai music21 pebble fluidsynth midi2audio

In [36]:
from midi2audio import FluidSynth
from IPython.display import Audio

In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [38]:
from IPython.display import Audio
from midi2audio import FluidSynth

In [ ]:
pip install mido

In [40]:
import glob
import pickle
from keras.layers import BatchNormalization as BatchNorm
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from mido import Message, MidiFile, MidiTrack, bpm2tempo, MetaMessage
import numpy as np

In [8]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
def train_network():

    """ This function calls all other functions and trains the LSTM"""

    # notes = get_notes()

    with open('/content/notes_trap', 'rb') as filepath:
        notes = pickle.load(filepath)

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [ ]:
def get_notes():
    notes = []

    for file in glob.glob("midi_drums/*.midi"):
        input_midi = MidiFile(file)

        for t in input_midi.tracks[-1]:
            # if 'track' in t.type:
            #     notes.append(t.type)
            if t.type == 'program_change':
                notes.append(f'{t.type},{t.program},{t.time}')
            elif t.type == 'control_change':
                notes.append(f'{t.type},{t.value},{t.time}')
            elif t.type == 'note_on' or t.type == 'note_off':
                notes.append(f'{t.type},{t.note},{t.time},{t.velocity}')
            else:
                continue
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    return notes

In [ ]:
def prepare_sequences(notes, n_vocab):

    """ Prepare the sequences which are the inputs for the LSTM """

    # sequence length should be changed after experimenting with different numbers
    sequence_length = 30

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [9]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        1024,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(1024, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(1024))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    model.load_weights('/content/weights-improvement-17-0.2038-bigger.hdf5')

    return model

In [ ]:
def train(model, network_input, network_output):

    """ train the neural network """

    filepath = "weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    # experiment with different epoch sizes and batch sizes
    model.fit(network_input, network_output, epochs=100, batch_size=512, callbacks=callbacks_list)

In [ ]:
train_network()

In [21]:
def generate(tempo=120):
    """ Generates the midi file """
    #load the notes used to train the model
    with open('/content/notes_trap', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output, tempo)
    create_wav()

In [22]:
def prepare_sequences(notes, pitchnames, n_vocab):

    """ Prepare the sequences used by the Neural Network """

    # map back from integers to notes
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 30
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [48]:
def generate_notes(model, network_input, pitchnames, n_vocab):

    """ Generate notes from the neural network based on a sequence of notes """

    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(800):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction) # numpy array of predictions
        result = int_to_note[index] # indexing the note with the highest probability
        prediction_output.append(result) # that note is the prediction output

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [42]:
def create_midi(prediction_output, tempo):
    mid = MidiFile()
    track = MidiTrack()
    track_meta = MidiTrack()
    mid.tracks.append(track_meta)
    mid.tracks.append(track)
    track_meta.append(MetaMessage('set_tempo', tempo=bpm2tempo(tempo)))
    track.append(Message('program_change', channel=9, program=12, time=0))

    for pattern in prediction_output:
        patterns = pattern.split(',')
        if patterns[0] == 'program_change':
            track.append(Message(patterns[0], channel=9, program=int(patterns[1]), time=int(patterns[2])))
        elif patterns[0] == 'control_change':
            track.append(Message(patterns[0], channel=9, value=int(patterns[1]), time=int(patterns[2])))
        else:
            track.append(Message(patterns[0], channel=9, note=int(patterns[1]), time=int(patterns[2]), velocity=int(patterns[3])))

    mid.save('/content/new_drums.mid')

In [43]:
def create_wav():
  fs = FluidSynth(sound_font='/content/808.sf2')
  fs.midi_to_audio('/content/new_drums.mid', '/content/output.wav')

In [49]:
generate(tempo=105)